## Pattern-Based Formulation and Column Generation

The previous roll-based ILP formulation is inefficient because:

- It contains many symmetric solutions (many equivalent ways to assign items to identical rolls), which makes branch-and-bound very slow.
- It has a large number of integer or binary variables.
- Even the LP relaxation (removing integrality) is hard because of the huge number of variables.

Column generation tackles this by **not enumerating all variables (columns) explicitly**. Instead, it maintains a **restricted master problem** with a small subset of patterns and dynamically adds new, promising patterns when needed.

A key observation from the simplex method:

- The number of basic variables (non-zero variables in a basic feasible solution) is at most the number of constraints.
- So, although the *potential* number of variables (columns) is huge, the optimal solution uses only a small subset.

Column generation exploits this by representing variables implicitly and generating new ones through a **pricing problem**.

---

### Pattern-Based Master Problem

Instead of deciding how each roll is cut, we now decide **how many times each pattern is used**.

- Let $x_j$ = number of times pattern $j$ is used.
- Let $a_{ij}$ = number of times item $i$ appears in pattern $j$.
- Let $b_i$ = demand of item $i$.
- Let $n$ = number of patterns currently enumerated in the master problem.

Then the (current) master problem is:

$$
\min \sum_{j=1}^n x_j
$$

subject to demand constraints:

$$
\sum_{j=1}^n a_{ij} x_j \ge b_i, \quad i = 1, 2, \dots, m
$$

and integrality:

$$
x_j \ge 0, \; x_j \in \mathbb{Z}, \quad j = 1, 2, \dots, n.
$$

We do **not** list all possible patterns (there are too many). Instead, we start with a small initial set (e.g., simple patterns) and then use column generation to add new patterns when they are profitable.

---

### Pricing Problem (Generating a New Pattern)

Let:

- $L$ = length of the large roll,
- $l_j$ = length of item $j$ (for $j = 1, \dots, m$),
- $z_j$ = number of times item $j$ appears in a *candidate* new pattern.

Feasibility of a pattern:

$$
\sum_{j=1}^m l_j z_j \le L, \quad z_j \ge 0, \; z_j \in \mathbb{Z}.
$$

Let $\pi_1, \dots, \pi_m$ be the dual variables associated with the demand constraints:

$$
\sum_{j=1}^n a_{ij} x_j \ge b_i, \quad i = 1, \dots, m.
$$

The reduced cost of a new column (pattern) defined by $z_j$ is:

$$
\text{rc} = 1 - \sum_{j=1}^m \pi_j z_j.
$$

- The coefficient in the objective of the master is $1$ (each roll used costs $1$).
- The dual contribution is $\sum_{j=1}^m \pi_j z_j$.

Since the master problem is a minimization, we want to find a new pattern with **negative reduced cost**. This is equivalent to solving:

$$
\max \sum_{j=1}^m \pi_j z_j
$$

subject to:

$$
\sum_{j=1}^m l_j z_j \le L,
$$

$$
z_j \ge 0, \; z_j \in \mathbb{Z}, \quad j = 1, 2, \dots, m.
$$

This is an **integer knapsack problem**, where:

- The "profit" of including item $j$ in the pattern is $\pi_j$,
- The "weight" is $l_j$,
- The capacity is $L$.

If the optimal value of this knapsack is greater than $1$, then the reduced cost

$$
\text{rc} = 1 - \sum_{j=1}^m \pi_j z_j
$$

is negative, and we have found a **new improving pattern** to add as a column to the master problem.

If no pattern yields $\sum_{j=1}^m \pi_j z_j > 1$, then all reduced costs are non-negative and the current LP solution of the master problem is optimal (for the LP relaxation).

The full column generation loop is then:

1. Solve the restricted master LP (with current patterns).
2. Extract dual values $\pi_j$.
3. Solve the knapsack pricing problem.
4. If a pattern with negative reduced cost exists, add it as a new column and go back to step 1.
5. If not, stop: the current master LP solution is optimal for the relaxation; then you may embed this inside branch-and-bound (branch-and-price) to restore integrality.


## Column Generation – Simple Illustrative Example

We consider the same cutting stock setting with:

- large roll length $L = 70$
- two item types:
  - item 1: length $l_1 = 17$
  - item 2: length $l_2 = 15$
- demands:
  - $b_1 = 8$
  - $b_2 = 6$

Instead of assigning cuts to each roll individually, we enumerate cutting **patterns**.

### Step 1: Initial Feasible Patterns

We start with a minimal pattern set, usually one pattern *per item type*:

- Pattern 1: only item 1  
  $a_{1,1} = \left\lfloor 70 / 17 \right\rfloor = 4$, so $4 \times 17 = 68$ with $2$ waste  
  $a_{2,1} = 0$

- Pattern 2: only item 2  
  $a_{1,2} = 0$  
  $a_{2,2} = \left\lfloor 70 / 15 \right\rfloor = 4$, so $4 \times 15 = 60$ with $10$ waste

Initial restricted master LP:

$$
\min x_1 + x_2
$$

subject to demand satisfaction:

$$
4 x_1 \ge 8
$$

$$
4 x_2 \ge 6
$$

and non-negativity:

$$
x_1, x_2 \ge 0.
$$

Solving this relaxation yields:

$$
x_1 = 2, \quad x_2 = 1.5,
$$

objective value:

$$
x_1 + x_2 = 3.5.
$$

This fractional solution is expected because the LP relaxation allows $x_j$ to be non-integer.

### Step 2: Extract Dual Values

From the LP solver, we retrieve duals associated with the demand constraints:

- dual for item 1 demand: $\pi_1$
- dual for item 2 demand: $\pi_2$

These dual values tell us how valuable it is to include each item type in a new pattern.

### Step 3: Pricing (Knapsack Subproblem)

To find a new improving pattern, solve:

$$
\max \pi_1 z_1 + \pi_2 z_2
$$

subject to:

$$
17 z_1 + 15 z_2 \le 70,
$$

$$
z_1, z_2 \ge 0, \; z_1, z_2 \in \mathbb{Z}.
$$

Interpretation:

- $z_1$ = number of items of type 1 in the new pattern
- $z_2$ = number of items of type 2 in the new pattern

This is an integer knapsack problem: choose $z_1, z_2$ to maximize profit given length capacity.

If the optimal value satisfies:

$$
\pi_1 z_1 + \pi_2 z_2 > 1,
$$

then reduced cost is:

$$
\text{rc} = 1 - (\pi_1 z_1 + \pi_2 z_2) < 0,
$$

meaning we found a **profitable new pattern** to add as a column in the restricted master problem.

### Step 4: Add New Pattern

Assume the knapsack returns:

- $z_1 = 3$, $z_2 = 1$
- Total used length: $3 \times 17 + 1 \times 15 = 66$
- Waste: $4$

This is a valid new pattern:

| Pattern | $a_{1j}$ | $a_{2j}$ | Waste |
|--------|----------|----------|--------|
| 3×17 + 1×15 | 3 | 1 | 4 |

We now enlarge the master problem with $x_3$ corresponding to this pattern.

### Step 5: Iterate

Solve the master LP again with the new column:

$$
\min x_1 + x_2 + x_3
$$

with:

- updated demand constraints
- new variable $x_3$

If no further negative reduced cost pattern exists, then:

- the LP relaxation is optimal for current patterns,
- next step is branching if integrality is required (branch-and-price).

---

### What This Demonstrates

- The full universe of patterns is **not** enumerated.
- Only patterns that improve the objective (negative reduced cost) are generated.
- This avoids billions of symmetric assignments.

Column generation turns the cutting stock problem from an impossible explicit enumeration task into a tractable iterative refinement process.

The core loop is:

1. solve restricted master LP
2. get duals
3. solve knapsack pricing problem
4. if reduced cost < 0 → add pattern and repeat
5. else stop

This simple example shows how one new pattern (3×17 + 1×15) can beat single-type patterns and improve solution quality while dramatically reducing the enumeration burden.



In [9]:
# %% [markdown]
# # Cutting Stock via Column Generation (Toy Example)
# 
# We implement a *simple* column generation algorithm for the cutting stock problem.
# 
# - Large roll length: L = 70
# - Item types:
#     - Item 1: length l1 = 17
#     - Item 2: length l2 = 15
# - Demands:
#     - b1 = 8
#     - b2 = 6
# 
# We:
# 1. Start from some initial patterns.
# 2. Solve the *restricted master problem* (LP).
# 3. Read dual variables π_i for the demand constraints.
# 4. Solve a knapsack *pricing problem* to find a new pattern.
# 5. Repeat until no pattern with negative reduced cost exists.
# 
# **Note:** To get dual values, we use GLPK through PuLP.
# You need `glpsol` installed on your system for this to work.


# %%
import pulp as pl

# Choose GLPK as solver (for duals). If GLPK is not installed,
# you can try CBC, but duals may not be available.
SOLVER = pl.PULP_CBC_CMD(msg=False, mip=False)


# %% [markdown]
# ## 1. Problem Data


# %%
# Large roll length
L = 70

# Item lengths (i -> l_i)
item_lengths = {
    1: 17,  # item 1
    2: 15,  # item 2
    3: 20   # need at least 6 pieces of length 15

}

# Demands (i -> b_i)
demands = {
    1: 8,   # demand for item 1
    2: 6,   # demand for item 2
    3: 7   # need at least 6 pieces of length 15

}

items = sorted(item_lengths.keys())
m = len(items)

print("L =", L)
print("item_lengths =", item_lengths)
print("demands =", demands)


# %% [markdown]
# ## 2. Representation of Patterns
# 
# A pattern is defined by how many items of each type it contains:
# 
# - For pattern p:
#     - `pattern[p][i]` = a_{i p} = number of times item i appears in pattern p.
# 
# We store patterns as a list of dicts:
# 
# ```python
# patterns = [
#     {1: 4, 2: 0},  # pattern 0
#     {1: 0, 2: 4},  # pattern 1
#     ...
# ]
# ```


# %%
def pattern_length(pattern):
    """Compute total used length of a pattern."""
    return sum(item_lengths[i] * pattern.get(i, 0) for i in items)


def print_patterns(patterns):
    """Nicely print the current patterns."""
    print("Current patterns:")
    for idx, pat in enumerate(patterns):
        used_len = pattern_length(pat)
        waste = L - used_len
        desc = ", ".join(
            f"{cnt} × item {i} (len {item_lengths[i]})"
            for i, cnt in pat.items()
            if cnt > 0
        ) or "empty"
        print(f"  Pattern {idx}: {desc} | used = {used_len}, waste = {waste}")
    print()


# %% [markdown]
# ## 3. Initial Patterns
# 
# A simple choice:
# 
# - Pattern 0: only item 1 (as many as possible)
# - Pattern 1: only item 2 (as many as possible)


# %%
patterns = []

# Pattern 0: only item 1
max_item1 = L // item_lengths[1]
patterns.append({1: max_item1, 2: 0})

# Pattern 1: only item 2
max_item2 = L // item_lengths[2]
patterns.append({1: 0, 2: max_item2})

# Pattern 1: only item 2
max_item3 = L // item_lengths[3]
patterns.append({1: 0, 2:0, 3: max_item3})

print_patterns(patterns)


# %% [markdown]
# ## 4. Restricted Master Problem (RMP)
# 
# Given a set of patterns, the RMP is:
# 
# Minimize:
# 
# $$ \sum_{p} x_p $$
# 
# subject to demand constraints:
# 
# $$ \sum_p a_{i p} x_p \ge b_i \quad \forall i $$
# 
# and:
# 
# $$ x_p \ge 0. $$
# 
# We solve it as an LP (continuous $x_p$), and then read:
# 
# - optimal pattern usage $x_p$,
# - dual variables $\pi_i$ for demand constraints.


# %%
def solve_master(patterns, solver=SOLVER, verbose=False):
    """
    Solve the restricted master LP for current patterns.
    
    Returns:
        x_values: dict pattern_index -> value
        duals: dict item_index -> dual value π_i
        obj_value: optimal objective value
    """
    prob = pl.LpProblem("CuttingStock_RMP", pl.LpMinimize)

    # Decision variables x_p >= 0 (continuous)
    pattern_indices = list(range(len(patterns)))
    x = pl.LpVariable.dicts("x", pattern_indices, lowBound=0, cat=pl.LpContinuous)

    # Objective: minimize total number of rolls
    prob += pl.lpSum(x[p] for p in pattern_indices), "Minimize_number_of_rolls"

    # Demand constraints: sum_p a_{i p} x_p >= b_i
    demand_constraints = {}
    for i in items:
        constraint = pl.lpSum(patterns[p].get(i, 0) * x[p] for p in pattern_indices) >= demands[i]
        cname = f"Demand_item_{i}"
        prob += constraint, cname
        demand_constraints[i] = prob.constraints[cname]

    # Solve
    prob.solve(solver)

    status = pl.LpStatus[prob.status]
    obj_value = pl.value(prob.objective)

    if verbose:
        print("RMP Status:", status)
        print("RMP Objective:", obj_value)

    # Extract primal solution x_p
    x_values = {p: x[p].value() for p in pattern_indices}

    # Extract duals π_i
    duals = {}
    for i in items:
        duals[i] = demand_constraints[i].pi

    if verbose:
        print("Pattern usage (x_p):")
        for p in pattern_indices:
            print(f"  x_{p} = {x_values[p]}")
        print("Dual values (π_i):")
        for i in items:
            print(f"  π_{i} = {duals[i]}")
        print()

    return x_values, duals, obj_value


# %% [markdown]
# ### Test the Master Problem on Initial Patterns


# %%
x_vals, duals, obj_val = solve_master(patterns, verbose=True)


# %% [markdown]
# ## 5. Pricing Problem (Knapsack)
# 
# Given duals $\pi_i$, we solve the unbounded knapsack:
# 
# $$ \max \sum_i \pi_i z_i $$
# 
# subject to:
# 
# $$ \sum_i l_i z_i \le L, $$
# 
# $$ z_i \ge 0, \; z_i \in \mathbb{Z}. $$
# 
# If the optimal value is $\le 1$, no improving pattern exists.
# If it is $> 1$, we have a pattern with **negative reduced cost**:
# 
# $$ \text{rc} = 1 - \sum_i \pi_i z_i < 0. $$
# 
# We solve this knapsack by dynamic programming and reconstruct $z$.


# %%
def solve_pricing_knapsack(duals):
    """
    Solve the unbounded knapsack pricing problem:
    
        max sum_i π_i z_i
        s.t. sum_i l_i z_i <= L, z_i ∈ Z_+
    
    Returns:
        best_value: optimal objective value
        pattern: dict i -> z_i (the new pattern)
    """
    # Capacity = L, weights = item_lengths[i], profits = duals[i]
    capacity = L

    # dp[c] = best profit with total length exactly c
    dp = [0.0] * (capacity + 1)
    # keep track of choice: (prev_capacity, item_chosen)
    choice = [None] * (capacity + 1)

    for c in range(capacity + 1):
        for i in items:
            w = item_lengths[i]
            p = duals[i]
            if c + w <= capacity:
                new_profit = dp[c] + p
                if new_profit > dp[c + w] + 1e-9:
                    dp[c + w] = new_profit
                    choice[c + w] = (c, i)

    # Find best profit over all capacities <= L
    best_value = max(dp)
    best_c = max(range(capacity + 1), key=lambda c: dp[c])

    # Reconstruct pattern z_i
    pattern = {i: 0 for i in items}
    c = best_c
    while c != 0 and choice[c] is not None:
        prev_c, i = choice[c]
        pattern[i] += 1
        c = prev_c

    return best_value, pattern


# %% [markdown]
# ### Test the Pricing Problem Once


# %%
best_value, new_pattern = solve_pricing_knapsack(duals)
print("Best value from pricing (Σ π_i z_i):", best_value)
print("Candidate new pattern z_i:", new_pattern)

reduced_cost = 1 - best_value
print("Reduced cost of new pattern:", reduced_cost)


# %% [markdown]
# ## 6. Column Generation Loop
# 
# We now put everything together:
# 
# 1. Start with initial patterns.
# 2. Solve RMP, get duals.
# 3. Solve pricing problem.
# 4. If best_value > 1 (reduced cost < 0):
#     - add the new pattern to the pattern list
#     - repeat
# 5. Else:
#     - stop (LP relaxation is optimal for the current set of patterns).
# 
# We limit the number of iterations to avoid infinite loops in case of issues.


# %%
def column_generation(
    initial_patterns,
    max_iterations=20,
    solver=SOLVER,
    verbose=True,
):
    patterns = list(initial_patterns)  # copy

    if verbose:
        print("=== Starting Column Generation ===")
        print_patterns(patterns)

    for it in range(1, max_iterations + 1):
        if verbose:
            print(f"--- Iteration {it} ---")

        # 1) Solve RMP
        x_vals, duals, obj_val = solve_master(patterns, solver=solver, verbose=verbose)

        # 2) Solve pricing
        best_value, new_pattern = solve_pricing_knapsack(duals)
        reduced_cost = 1 - best_value

        if verbose:
            print("Pricing result:")
            print("  best Σ π_i z_i =", best_value)
            print("  reduced cost =", reduced_cost)
            print("  candidate pattern:", new_pattern)
            print()

        # 3) Check termination
        if best_value <= 1 + 1e-6:
            if verbose:
                print("No pattern with negative reduced cost found.")
                print("Column generation for LP relaxation has converged.")
            break

        # 4) Add new pattern
        patterns.append(new_pattern)
        if verbose:
            print("New pattern added.")
            print_patterns(patterns)

    # Final solve to get final LP solution
    x_vals, duals, obj_val = solve_master(patterns, solver=solver, verbose=False)

    if verbose:
        print("=== Final LP Solution ===")
        print("Objective (min rolls) =", obj_val)
        print("Pattern usage (x_p):")
        for p, val in x_vals.items():
            print(f"  Pattern {p}: x_{p} = {val}")
        print()
        print("Patterns:")
        print_patterns(patterns)

    return patterns, x_vals, duals, obj_val


# %% [markdown]
# ### Run Column Generation


# %%
final_patterns, final_x, final_duals, final_obj = column_generation(patterns)


L = 70
item_lengths = {1: 17, 2: 15, 3: 20}
demands = {1: 8, 2: 6, 3: 7}
Current patterns:
  Pattern 0: 4 × item 1 (len 17) | used = 68, waste = 2
  Pattern 1: 4 × item 2 (len 15) | used = 60, waste = 10
  Pattern 2: 3 × item 3 (len 20) | used = 60, waste = 10

RMP Status: Optimal
RMP Objective: 5.8333333
Pattern usage (x_p):
  x_0 = 2.0
  x_1 = 1.5
  x_2 = 2.3333333
Dual values (π_i):
  π_1 = 0.25
  π_2 = 0.25
  π_3 = 0.33333333

Best value from pricing (Σ π_i z_i): 1.16666666
Candidate new pattern z_i: {1: 0, 2: 2, 3: 2}
Reduced cost of new pattern: -0.16666665999999997
=== Starting Column Generation ===
Current patterns:
  Pattern 0: 4 × item 1 (len 17) | used = 68, waste = 2
  Pattern 1: 4 × item 2 (len 15) | used = 60, waste = 10
  Pattern 2: 3 × item 3 (len 20) | used = 60, waste = 10

--- Iteration 1 ---
RMP Status: Optimal
RMP Objective: 5.8333333
Pattern usage (x_p):
  x_0 = 2.0
  x_1 = 1.5
  x_2 = 2.3333333
Dual values (π_i):
  π_1 = 0.25
  π_2 = 0.25
  π_3 = 0.33333333

Pric